In [2]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
import csv
import cv2
import numpy as np
from scipy import misc
from keras_vggface import utils
from keras.preprocessing import image
from sklearn import preprocessing
from scipy.spatial import distance
import os
import pandas as pd

In [3]:
from mtcnn.mtcnn import MTCNN

In [8]:
def l2_norm(x):
    #x2=tf.nn.l2_normalize(x,axis=None,epsilon=1e-12,name=None,dim=None)
    x2=tf.nn.l2_normalize(x,dim=1)
    return x2


# CHOOSE AN ARCHITECTURE:  
#architecture = 'vgg'
#folderName = 'VGG1/'

architecture = 'resnet'
folderName = 'VGG2/'

dirpath = os.getcwd()

# The images that we are using are already cropped, 
# so we don't need to detect face positions

bbox_1 = [0,0,224,224]
bbox_2 = [0,0,224,224]


if architecture == 'vgg':
    #baseline model with vgg:
    vgg_model = VGGFace(model = 'vgg16')
#                        CHANGE LAYER HERE  \/
    feature_layer = vgg_model.get_layer('fc7/relu').output
    model = Model(vgg_model.input, feature_layer)
    version = 1 


elif architecture == 'resnet':
    #baseline model with resnet:
    resnet = VGGFace(model = 'resnet50')
    
#                   CHANGE LAYER HERE  \/
    last_layer = resnet.get_layer('avg_pool').output
    feature_layer = Flatten(name='flatten')(last_layer)
    model = Model(resnet.input, feature_layer)
    version = 2   


In [25]:
#Download Models:

# keras.backend.set_image_dim_ordering('tf')

# keras.backend.image_data_format('tf')

# VGG1:
#model = VGGFace(model='vgg16', include_top=False) #include_top=False to only extract features

# VGG2
#model = VGGFace(model='resnet50', include_top=False) #include_top=False to only extract features

In [11]:
Database = 'Cropped/ImageSamples/Averages/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open((folderName+'features_averages.csv'),'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_averages.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_averages.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_averages.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_averages.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_averages.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_averages.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

averageFace_1_1_100_1.jpg
averageFace_1_1_100_10.jpg
averageFace_1_1_100_2.jpg
averageFace_1_1_100_3.jpg
averageFace_1_1_100_4.jpg
averageFace_1_1_100_5.jpg
averageFace_1_1_100_6.jpg
averageFace_1_1_100_7.jpg
averageFace_1_1_100_8.jpg
averageFace_1_1_100_9.jpg
averageFace_1_1_10_1.jpg
averageFace_1_1_10_10.jpg
averageFace_1_1_10_2.jpg
averageFace_1_1_10_3.jpg
averageFace_1_1_10_4.jpg
averageFace_1_1_10_5.jpg
averageFace_1_1_10_6.jpg
averageFace_1_1_10_7.jpg
averageFace_1_1_10_8.jpg
averageFace_1_1_10_9.jpg
averageFace_1_1_110_1.jpg
averageFace_1_1_110_10.jpg
averageFace_1_1_110_2.jpg
averageFace_1_1_110_3.jpg
averageFace_1_1_110_4.jpg
averageFace_1_1_110_5.jpg
averageFace_1_1_110_6.jpg
averageFace_1_1_110_7.jpg
averageFace_1_1_110_8.jpg
averageFace_1_1_110_9.jpg
averageFace_1_1_120_1.jpg
averageFace_1_1_120_10.jpg
averageFace_1_1_120_2.jpg
averageFace_1_1_120_3.jpg
averageFace_1_1_120_4.jpg
averageFace_1_1_120_5.jpg
averageFace_1_1_120_6.jpg
averageFace_1_1_120_7.jpg
averageFace_1_1_12

KeyboardInterrupt: 

In [11]:
Database = 'MorphDatabase/1_1/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open((folderName+'features_1_1.csv'),'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_1_1.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_1_1.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_1_1.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_1_1.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_1_1.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_1_1.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

030380_0M64.JPG
030380_1M64.JPG
034295_0M54.JPG
034295_1M54.JPG
035143_0M56.JPG
035143_1M56.JPG
035557_01M53.JPG
035557_02M54.JPG
037372_02M54.JPG
037372_1M52.JPG
038292_02M57.JPG
038292_0M54.JPG
038566_0M55.JPG
038566_1M56.JPG
038836_0M52.JPG
038836_1M53.JPG
041985_0M49.JPG
041985_1M50.JPG
042432_0M52.JPG
042432_1M53.JPG
042778_00M52.JPG
042778_01M53.JPG
045501_00M48.JPG
045501_01M49.JPG
045556_00M55.JPG
045556_02M55.JPG
046095_0M45.JPG
046095_1M47.JPG
046666_02M51.JPG
046666_03M51.JPG
046770_2M45.JPG
046770_4M45.JPG
046925_00M50.JPG
046925_01M50.JPG
046970_0M46.JPG
046970_1M47.JPG
047373_0M46.JPG
047373_1M46.JPG
048703_0M45.JPG
048703_6M47.JPG
049193_03M56.JPG
049193_2M54.JPG
049268_0M47.JPG
049268_1M47.JPG
049679_03M48.JPG
049679_04M48.JPG
049754_0M46.JPG
049754_1M46.JPG
050405_0M45.JPG
050405_1M46.JPG
050671_0M43.JPG
050671_1M44.JPG
051283_1M47.JPG
051283_2M48.JPG
051325_11M44.JPG
051325_3M43.JPG
051383_0M46.JPG
051383_1M46.JPG
052022_02M54.JPG
052022_0M53.JPG
054134_0M50.JPG
05413

In [9]:
Database = 'MorphDatabase/1_2/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open(folderName+'features_1_2.csv','a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_1_2.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_1_2.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_1_2.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_1_2.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_1_2.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_1_2.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

026988_0M59.JPG
026988_1M61.JPG
028580_0M60.JPG
028580_1M60.JPG
030624_0M63.JPG
030624_1M64.JPG
036182_1M53.JPG
036182_2M54.JPG
037171_0M54.JPG
037171_1M55.JPG
038211_02M59.JPG
038211_0M55.JPG
039820_2M49.JPG
039820_4M51.JPG
040007_00M53.JPG
040007_02M53.JPG
040611_1M48.JPG
040611_2M49.JPG
040768_0M53.JPG
040768_1M54.JPG
041430_0M56.JPG
041430_1M56.JPG
041813_0M50.JPG
041813_2M52.JPG
042647_02M50.JPG
042647_1M50.JPG
042663_0M56.JPG
042663_1M57.JPG
044379_1M50.JPG
044379_2M50.JPG
044653_0M46.JPG
044653_1M46.JPG
045849_1M47.JPG
045849_3M48.JPG
046747_0M50.JPG
046747_1M51.JPG
047304_01M48.JPG
047304_02M48.JPG
047427_0M47.JPG
047427_1M47.JPG
047564_3M21.JPG
047564_5M54.JPG
047712_0M45.JPG
047712_1M46.JPG
047818_00M49.JPG
047818_02M50.JPG
047984_02M57.JPG
047984_0M54.JPG
048157_0M52.JPG
048157_1M54.JPG
048521_0M44.JPG
048521_1M46.JPG
049077_0M49.JPG
049077_1M49.JPG
049099_1M44.JPG
049099_2M46.JPG
049114_0M47.JPG
049114_1M47.JPG
049149_00M55.JPG
049149_01M53.JPG
050979_0M43.JPG
050979_15M45.

In [10]:
Database = 'MorphDatabase/2_1/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open(folderName+'features_2_1.csv','a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_2_1.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_2_1.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_2_1.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_2_1.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_2_1.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_2_1.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

046763_00F49.JPG
046763_01F45.JPG
047697_0F47.JPG
047697_2F48.JPG
047840_1F55.JPG
047840_3F56.JPG
048125_10F48.JPG
048125_8F48.JPG
050839_0F43.JPG
050839_2F44.JPG
052819_03F46.JPG
052819_0F42.JPG
052858_0F45.JPG
052858_1F46.JPG
053194_1F51.JPG
053194_2F51.JPG
053966_0F46.JPG
053966_3F48.JPG
054186_0F49.JPG
054186_1F49.JPG
055507_1F47.JPG
055507_2F48.JPG
055734_2F44.JPG
055734_7F45.JPG
057065_0F45.JPG
057065_2F45.JPG
057521_0F40.JPG
057521_2F41.JPG
057605_0F43.JPG
057605_1F43.JPG
059534_0F41.JPG
059534_1F41.JPG
060859_0F52.JPG
060859_1F53.JPG
061169_0F41.JPG
061169_1F41.JPG
061973_0F40.JPG
061973_1F41.JPG
062234_02F45.JPG
062234_03F43.JPG
063379_1F43.JPG
063379_4F43.JPG
064999_02F45.JPG
064999_1F42.JPG
065369_3F41.JPG
065369_4F41.JPG
066835_0F45.JPG
066835_1F47.JPG
067050_0F46.JPG
067050_1F46.JPG
068130_0F74.JPG
068130_1F75.JPG
068353_1F40.JPG
068353_2F41.JPG
068423_2F39.JPG
068423_4F39.JPG
068453_01F39.JPG
068453_02F40.JPG
069524_0F58.JPG
069524_3F59.JPG
070362_0F37.JPG
070362_1F37.JPG

In [11]:
Database = 'MorphDatabase/2_2/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open(folderName+'features_2_2.csv','a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_2_2.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_2_2.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_2_2.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_2_2.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_2_2.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_2_2.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

039661_0F59.JPG
039661_2F61.JPG
042379_1F49.JPG
042379_3F49.JPG
042667_0F50.JPG
042667_2F51.JPG
043709_3F49.JPG
043709_5F50.JPG
046529_0F52.JPG
046529_1F53.JPG
049795_0F48.JPG
049795_1F48.JPG
050146_0F62.JPG
050146_1F62.JPG
050750_0F43.JPG
050750_2F44.JPG
051879_2F53.JPG
051879_3F54.JPG
052362_0F46.JPG
052362_1F46.JPG
052417_0F44.JPG
052417_1F45.JPG
052740_00F46.JPG
052740_01F46.JPG
052855_02F49.JPG
052855_1F47.JPG
052863_0F48.JPG
052863_2F50.JPG
054065_0F48.JPG
054065_1F49.JPG
054825_0F47.JPG
054825_1F48.JPG
054980_0F45.JPG
054980_1F45.JPG
055873_0F45.JPG
055873_1F46.JPG
056125_1F43.JPG
056125_2F43.JPG
056736_1F48.JPG
056736_2F49.JPG
056743_0F50.JPG
056743_1F50.JPG
057353_0F48.JPG
057353_1F48.JPG
057566_0F49.JPG
057566_1F50.JPG
058106_03F48.JPG
058106_2F46.JPG
058404_1F42.JPG
058404_2F44.JPG
058515_00F44.JPG
058515_01F45.JPG
059159_1F42.JPG
059159_3F44.JPG
060702_05F47.JPG
060702_4F45.JPG
060922_0F40.JPG
060922_2F42.JPG
061002_5F44.JPG
061002_6F44.JPG
062103_00F45.JPG
062103_01F45.JPG

In [34]:
Database = 'keyAverages/Morphs/'
################################################################


list_dir = os.listdir(Database)
list_dir = [f for f in list_dir]
list_dir = sorted(list_dir)
list_dir
# Extract Features:
for i in range (0,len(list_dir)):
    img = cv2.imread(Database + list_dir[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
###############
#     crop_img = crop(img)
    crop_img = img
###############


    x = image.img_to_array(img)
    
    img = image.load_img(Database + list_dir[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    preds = model.predict(x)
# Save as a csv file:
    with open(folderName+'features_morph.csv','a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(preds[0])
    print('{}'.format(list_dir[i]))
print("DONE.")

# Preprocess CSV file to save as a concatenated version of features.csv:
print('Saving File...')
oi = pd.read_csv(folderName+'features_morph.csv',header=None)
hi = np.array(oi)
np.savetxt(folderName+'Concatenated_Features_morph.csv',hi,delimiter = ',')
print('Finished.')

############ EUCLIDEAN DISTANCE AREA ################

db1 = pd.read_csv(folderName+'Concatenated_Features_morph.csv',header = None, sep=',')
db2 = pd.read_csv(folderName+'Concatenated_Features_morph.csv',header = None, sep=',')

### Transform into array:

db1_array = np.array(db1)
db2_array = np.array(db2)

np.shape(db1_array)

### Calculate euclidian distances:

DistanceVector = []
for i in range (0,len(db1_array[:,0])): #Number of rows
    x = db1_array[i,:]
    for j in range (0,len(db2_array[:,0])): #Number of rows
        y = db2_array[j,:]
        DistanceVector.append(np.linalg.norm(x-y))
#     np.savetxt('Ouclidean_distances_TRY.csv',DistanceVector,delimiter = ',')
    print('Image {}'.format(i+1))

exp = DistanceVector
exp = np.reshape(exp,[len(db1_array[:,0]),len(db2_array[:,0])])
print('Saving File...')
np.savetxt(folderName+'Euclidean_Distances_morph.csv',exp,delimiter=',')
print('Finished.')
with open('filenames_morph.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)

morph_1_10.jpg
morph_1_20.jpg
morph_1_30.jpg
morph_1_40.jpg
morph_1_50.jpg
morph_1_60.jpg
morph_1_70.jpg
morph_1_80.jpg
morph_1_90.jpg
morph_2_10.jpg
morph_2_20.jpg
morph_2_30.jpg
morph_2_40.jpg
morph_2_50.jpg
morph_2_60.jpg
morph_2_70.jpg
morph_2_80.jpg
morph_2_90.jpg
morph_3_10.jpg
morph_3_20.jpg
morph_3_30.jpg
morph_3_40.jpg
morph_3_50.jpg
morph_3_60.jpg
morph_3_70.jpg
morph_3_80.jpg
morph_3_90.jpg
morph_4_10.jpg
morph_4_20.jpg
morph_4_30.jpg
morph_4_40.jpg
morph_4_50.jpg
morph_4_60.jpg
morph_4_70.jpg
morph_4_80.jpg
morph_4_90.jpg
testFace1.jpg
testFace2.jpg
testFace3.jpg
testFace4.jpg
DONE.
Saving File...
Finished.
Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
Image 10
Image 11
Image 12
Image 13
Image 14
Image 15
Image 16
Image 17
Image 18
Image 19
Image 20
Image 21
Image 22
Image 23
Image 24
Image 25
Image 26
Image 27
Image 28
Image 29
Image 30
Image 31
Image 32
Image 33
Image 34
Image 35
Image 36
Image 37
Image 38
Image 39
Image 40
Saving File...
Finishe

In [33]:
################################################################
Database = 'Cropped/1_1/'

list_dir = os.listdir(Database)

with open('filenames_1_1.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)
    
    
################################################################
Database = 'Cropped/1_2/'

list_dir = os.listdir(Database)

with open('filenames_1_2.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)
    
################################################################
Database = 'Cropped/2_1/'

list_dir = os.listdir(Database)

with open('filenames_2_1.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)
    
################################################################
Database = 'Cropped/2_2/'

list_dir = os.listdir(Database)

with open('filenames_2_2.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)